Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load datasets

In [2]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

Separate the labels from the features and remove columns we dont need

In [3]:
# separate labels from features
training_label_cols = ['TotalTimeStopped_p20','TotalTimeStopped_p40','TotalTimeStopped_p50',
                       'TotalTimeStopped_p60','TotalTimeStopped_p80',
                       'TimeFromFirstStop_p20','TimeFromFirstStop_p40','TimeFromFirstStop_p50',
                       'TimeFromFirstStop_p60','TimeFromFirstStop_p80',
                       'DistanceToFirstStop_p20','DistanceToFirstStop_p40','DistanceToFirstStop_p50',
                       'DistanceToFirstStop_p60','DistanceToFirstStop_p80']

df_train_Y = df_train[training_label_cols]

# drop columns not needed
df_test_X = df_test.drop(columns=['RowId'])
df_train_X = df_train.drop(columns=['RowId']+training_label_cols)

Separate the Data by City

In [4]:
cities = ['Atlanta','Philadelphia','Boston','Chicago']
city_training_datasets = {}
city_testing_datasets = {}

# training
for city in cities:
    city_training_datasets[city] = df_train_X.loc[df_train_X['City'] == city]
    city_training_datasets[city] = city_training_datasets[city].drop(columns=['City'])

# testing
for city in cities:
    city_testing_datasets[city] = df_test_X.loc[df_test_X['City'] == city]
    city_testing_datasets[city] = city_testing_datasets[city].drop(columns=['City'])